# Проект в Портфолио. Музыкальные предпочтения

**Дано** - датасет со статистикой прослушивания музыки в Москве и Сантк-Петербурге. Датасет был основан на статистике одного известного музыкального сервиса. 

**Цель исследования** — проверить три гипотезы:
1. Активность пользователей зависит от дня недели и от города;
2. В понедельник утром и в пятницу вечером в Москве преобладают одни жанры, а в Петербурге — другие. 
3. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования:**

Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

In [76]:
# импорт библиотеки pandas
import pandas as pd

# чтение файла с данными и сохранение в df
df = pd.read_csv ('yandex_music_project.csv')

In [77]:
# рассмотрим первые 10 строк таблицы df
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Документация к данным:
* `userID` — идентификатор пользователя;
* `Track` — название трека;  
* `artist` — имя исполнителя;
* `genre` — название жанра;
* `City` — город пользователя;
* `time` — время начала прослушивания;
* `Day` — день недели.

In [78]:
# получим общую информацию о таблице df
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


In [79]:
# получим общую информацию о таблице df
df.describe()

,userID,Track,artist,genre,City,time,Day
count,65079,63848,57876,63881,65079,65079,65079
unique,41748,47245,43605,289,2,20392,3
top,A8AE9169,Intro,Sasha,pop,Moscow,08:14:07,Friday
freq,76,34,6,8850,45360,14,23149


**Выводы**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку.

Данные нужно предобработать:
* в данных встречаются пропуски;
* возможнои имеются дубликаты;
* названия столбцов необходимо преобразовать к нижнему регистру.

## Предобработка данных

### Стиль заголовков
Приведём названия стобцов к нижнему регистру

In [80]:
# перечень названий столбцов таблицы df
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [81]:
# переименование столбцов
df.columns = ['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day']

# альтернативный способ
# df = df.rename (columns = {'  userID':'user_id', 'Track':'track', '  City  ':'city', 'Day':'day'})

In [82]:
# проверка результатов 
df.columns 

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

Столбцы переименованы. Теперь работать удобнее

### Пропуски значений
Подсчитаем количество пропущенных значений для каждого столбца

In [83]:
# подсчёт пропусков
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

В зависимости от пропусков и от задачи можно:
* удалить строки с пропусками методом dropna()
* заполнить пропуски методом fillna() - например наиболее частым значением или строкой "unknown"
* оставить без изменений 

В данном исследовании заменим пропущенные значения в столбцах `track`, `artist` и `genre` на строку `'unknown'`

In [84]:
# создадим цикл для замены пропущенных значений на 'unknown'
columns_to_replace = ['track', 'artist', 'genre'] # столбцы для замены пропусков
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown') # заполняем методом fillna()

# результат
df.isna().sum() 

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

Пропущенные значения заполнены

### Дубликаты
Подсчитаем явные дубликаты строк (полностью идентичные строки)

In [85]:
# подсчёт явных дубликатов
ammount_dulicates = df.duplicated().sum()
print ('Количесвто явных дубликатов: ', ammount_dulicates)

Количесвто явных дубликатов:  3826


Дубликаты могут исказить результаты исследования. Их необходимо удалить.

In [86]:
# удаление явных дубликатов 
# с удалением старых индексов и формированием новых
df = df.drop_duplicates().reset_index (drop = True)

# результат
ammount_dulicates = df.duplicated().sum()
print ('Количесвто явных дубликатов после удаления: ', ammount_dulicates)

Количесвто явных дубликатов после удаления:  0


До удаления дубликатов - 65079 строк.

После удаления дубликатов - 61253 строки. 

Удаление явных дубликатов прошло успешно. Однако в столбцах могут быть записаны одни и те же значения с разным регистром, например. Назовём такие дубликаты неявными. От них тоже необходимо избавиться.

Рассмотрим уникальные названий жанров, отсортированный в алфавитном порядке.

In [87]:
# Просмотр уникальных названий жанров
sorted_gerne_series = df['genre'].sort_values().unique() 
print ('Количество уникальных жанров до удаления дубликатов: ', len (sorted_gerne_series))
sorted_gerne_series

Количество уникальных жанров до удаления дубликатов:  290


array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Были обнаружены следующие дубликаты:
- hip-hop ('hip', 'hop') .
- latin ('latin', 'latino');
- turkey ('türk', 'türkçe');
- электроника ('electronic').

Приведём данные жанры к единообразию.

In [88]:
# Создадим функция для переименования
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace (wrong_genre, correct_genre)

In [89]:
# вызовем функцию для каждого жанра 
# 'hiphop'
wrong_genres = ['hip', 'hip-hop', 'hop']
correct_genre = 'hiphop'            
replace_wrong_genres(wrong_genres, correct_genre)

# latin ('latin', 'latino');
wrong_genres = ['latin']
correct_genre = 'latino'
replace_wrong_genres(wrong_genres, correct_genre)

# turkey ('türk', 'türkçe');
wrong_genres = ['türk', 'türkçe']
correct_genre = 'turkey'
replace_wrong_genres(wrong_genres, correct_genre)

# электроника ('electronic').
wrong_genres = ['электроника']
correct_genre = 'electronic'              
replace_wrong_genres(wrong_genres, correct_genre)

In [90]:
# Проверка на неявные дубликаты после удаления 
sorted_gerne_series_clean = df['genre'].sort_values().unique() 
print ('Количество уникальных жанров до удаления дубликатов: ', len (sorted_gerne_series))
print ('Количество уникальных жанров после удаления дубликатов: ', len (sorted_gerne_series_clean))
sorted_gerne_series_clean

Количество уникальных жанров до удаления дубликатов:  290
Количество уникальных жанров после удаления дубликатов:  284


array(['acid', 'acoustic', 'action', 'adult', 'africa', 'afrikaans',
       'alternative', 'alternativepunk', 'ambient', 'americana',
       'animated', 'anime', 'arabesk', 'arabic', 'arena',
       'argentinetango', 'art', 'audiobook', 'author', 'avantgarde',
       'axé', 'baile', 'balkan', 'beats', 'bigroom', 'black', 'bluegrass',
       'blues', 'bollywood', 'bossa', 'brazilian', 'breakbeat', 'breaks',
       'broadway', 'cantautori', 'cantopop', 'canzone', 'caribbean',
       'caucasian', 'celtic', 'chamber', 'chanson', 'children', 'chill',
       'chinese', 'choral', 'christian', 'christmas', 'classical',
       'classicmetal', 'club', 'colombian', 'comedy', 'conjazz',
       'contemporary', 'country', 'cuban', 'dance', 'dancehall',
       'dancepop', 'dark', 'death', 'deep', 'deutschrock', 'deutschspr',
       'dirty', 'disco', 'dnb', 'documentary', 'downbeat', 'downtempo',
       'drum', 'dub', 'dubstep', 'eastern', 'easy', 'electronic',
       'electropop', 'emo', 'entehno', '

Неявные дубликаты удалены успешно

**Выводы**

Предобработка устранила три проблемы в данных:

- нарушения в стиле заголовков;
- заменены пропущенные значения;
- удалены дубликаты — явные и неявные.

Переходим к проверке гипотез. 

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Гипотеза 1 - Активность пользователей зависит от дня недели и от города.

Сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [91]:
# Подсчёт прослушиваний в каждом городе
group_by_city = df.groupby('city')['track'].count()
group_by_city 

city
Moscow              42741
Saint-Petersburg    18512
Name: track, dtype: int64

В Москве прослушиваний больше, чем в Петербурге. Из этого не следует, что московские пользователи чаще слушают музыку. Просто самих пользователей в Москве больше.

Теперь сгруппируем данные по дню недели и подсчитайте прослушивания в понедельник, среду и пятницу. 

In [92]:
# Подсчёт прослушиваний в каждый из трёх дней
group_by_day = df.groupby('day')['track'].count()
group_by_day

day
Friday       21840
Monday       21354
Wednesday    18059
Name: track, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но картина может измениться, если рассмотреть каждый город в отдельности.

Напишем функцию, которая объединит эти два расчёта.

In [93]:
# Функция для подсчёта прослушиваний для конкретного города и дня
# создание функции number_tracks()
def number_tracks(day, city):  # Объявляется функция с двумя параметрами: day, city.
    track_list = df[(df['day'] == day) & (df['city'] == city)]
    track_list_count = track_list['user_id'].count()
    return track_list_count

Вызовем функцию для каждого горада и дня недели

In [94]:
# передадим в функцию следующие параметры:
# список городов
cities = ['Moscow', 'Saint-Petersburg']
# список дней недели
days = ['Monday', 'Wednesday', 'Friday']

# создадим пустой список для сохраненния результатов в нём
data_set_for_days = [] 

for day in days:
    # создадим пустой список для записи промежуточных результатов для каждого дня недели
    list_for_city = [] 
    for city in cities:    
        list_for_city.append (number_tracks(day, city))
    data_set_for_days.append (list_for_city) 

data_set_for_days

[[15740, 5614], [11056, 7003], [15945, 5895]]

Создадим c помощью конструктора `pd.DataFrame` таблицу
* названия колонок — `['city', 'monday', 'wednesday', 'friday']`;
* данные — результаты, которые мы получили с помощью `number_tracks`.

In [95]:
# Таблица с результатами
data_city_day = pd.DataFrame (data = data_set_for_days, columns = cities, index=days)

# выведем на экран транспонированную таблицу
data_city_day.T 

,Monday,Wednesday,Friday
Moscow,15740,11056,15945
Saint-Petersburg,5614,7003,5895


**Выводы**

Из таблицы мы видим:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Первая гипотеза подвтреждена. Активность пользователей зависит от дня недели. Причём для Москвы и Санкт-Петербурга это проявляется по-разному.

### Музыка в начале и в конце недели

Гипотеза 2 - В понедельник утром и в пятницу вечером в Москве преобладают одни жанры, а в Петербурге — другие. 

Для проверки гипотезы создадим функцию, которая возвращает информацию о топ-10 жанров тех треков, которые прослушивали в указанный день, в промежутке между двумя отметками времени в данном городе. 

In [96]:
# из общей таблицы df отфильтруем строки Мск и Спб

moscow_general = df[df['city'] == 'Moscow'] # строки для Мск
spb_general = df[df['city'] == 'Saint-Petersburg'] # строки для Спб

In [97]:
# создадим функцию
def genre_weekday(dataframe, day, time_1, time_2): 
    """возвращает информацию о первых 10 жанрах тех треков, 
    которые прослушивали в день "day", в промежутке между 
    двумя отметками времени 'time_1' и 'time_2'.
    dataframe для искомого города задаётся предварительно отфильтрованый"""
    # фильтрация по дню недели
    genre_df = dataframe[dataframe['day'] == day]  
    # фильтрация по времени
    genre_df = genre_df [(genre_df['time'] > time_1) & (genre_df['time'] < time_2)]
    # группировка по жанру и подсчёт количесвта треков
    genre_df_count = genre_df.groupby('genre')['track'].count() 
    # сортировка результата 
    genre_df_count_sorted =  genre_df_count.sort_values(ascending = False) 
    return genre_df_count_sorted [:10]   

Выведем на экран результаты для двух городов 

In [98]:
# Мсоква. Понедельник. Утро
display ('Мсоква. Понедельник. Утро', genre_weekday (moscow_general, 'Monday', '07:00', '11:00'))

# Спб. Понедельник. Утро
display ('Спб. Понедельник. Утро', genre_weekday (spb_general, 'Monday', '07:00', '11:00'))

'Мсоква. Понедельник. Утро'

genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: track, dtype: int64

'Спб. Понедельник. Утро'

genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: track, dtype: int64

In [99]:
# Мсоква. Пятница. Вечер
display ('Мсоква. Пятница. Вечер', genre_weekday (moscow_general, 'Friday', '17:00', '23:00'))

# Спб. Пятница. Вечер
display ('Спб. Пятница. Вечер', genre_weekday (spb_general, 'Friday', '17:00', '23:00'))

'Мсоква. Пятница. Вечер'

genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: track, dtype: int64

'Спб. Пятница. Вечер'

genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: track, dtype: int64

**Выводы**

Исходя из таблицы, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза 3 - В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

Для проверки гипотезы сгруппируем таблицу `moscow_general` по жанру и посчитаем прослушивания треков каждого жанра методом `count()`.

Затем отсортируем результат в порядке убывания

In [100]:
# сгруппируем данные
moscow_genres = moscow_general.groupby('genre')['genre'].count().sort_values(ascending = False)

# выведем на экран первые десять строк `moscow_genres`
moscow_genres[:10]

genre
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: genre, dtype: int64

Повторим то же и для Петербурга

In [101]:
# сгруппируем данные
spb_genres = spb_general.groupby('genre')['genre'].count().sort_values(ascending = False)

# просмотр первых 10 строк spb_genres
spb_genres[:10]

genre
pop            2431
dance          1932
rock           1879
electronic     1737
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: genre, dtype: int64

**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Рэп одинаково популярен в Москве и Петербурге. 


## Дополнительное исследование данных

Исключительно из "спортивного" интереса мне стало любопытно, у кого наибольшее количество уникальных треков в плей листе. 

Выведем для каждого пользователя список уникальных треков

In [102]:
# сгруппируем данные по 'user_id' 
# и выведем для каждого пользователя список и количество уникальных треков
track_count = df.groupby('user_id').agg({'track': ['unique', 'nunique', 'count']})

# отсортируем по количеству уникальных треков
track_count.sort_values(by = ('track','nunique'), ascending = False)

track              
                                                     unique nunique count
user_id                                                                  
E8339398  [Don't Push Me, Negative=Positive, Backbeat, L...      43    43
D6A03F1E  [909 Drop, Is Stereo!, I Just Can't Stop, Rode...      35    36
B5496034  [Jealous Man, Think of You, Hell or Heaven, Eq...      34    34
B851E1A5  [No me Doy por Vencido, Mermaid Summer, Ogni g...      33    33
3D4399B4  [В конце пути, Офицеры (От героев былых времён...      29    29
...                                                     ...     ...   ...
6B084F4B                      [The Last Race (Death Proof)]       1     1
6B073DDC                                        [Back Down]       1     1
6B063F34                         [Instrumentāla Starpspēle]       1     1
6B044A0A                                        [The Shore]       1     1
FFFF6AC5                                    [Tika's Groove]       1     1

[41748 rows x 3 columns]

Самое большое количество прослушиваний уникальных треков - 43. И каждый по одному разу. Отсортировав по count() можно увидеть, что есть те, кто слушает 3 трека в общей сложности 50 раз.

Ещё немного поизучаем данные с точки зрения уникальных треков и количества прослушиваний

In [103]:
# длина списка уникальных треков для каждого пользователя
track_unique_ammount = track_count['track']['unique'].apply (len)

# суммарное количество уникальных треков для всех пользователей
track_unique_sum = track_unique_ammount.sum() 

# общее количество прослушиваний для всех пользователей 
track_all = track_count['track']['count'].sum()


# создадим табличку для отображения 
# в переменную data_df запишем полученные значения  
data_df = [track_unique_sum, track_all] 

# названия строк
index_df = ['Уникальные треки', 'Прослушивания']  

# создание таблички
unique_df = pd.DataFrame (data = data_df, index = index_df, columns = ['Количество'])
unique_df

,Количество
Уникальные треки,59573
Прослушивания,61253


Всего уникальных треков 59573, из прослушали 61253 - в среднем 1,028 раза каждый трек

## Итоги исследования

В ходе исследования было сделано:
1. Получена общая информация о данных, выявлены основные проблемыс(пропуски, дубликаты)
2. Данные предобработаны
3. Были проверены три гипотезы и установлено:

 -  День недели по-разному влияет на активность пользователей в Москве и Петербурге. 

 -  Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург.  Небольшие различия заметны в начале недели, по понедельникам.
 -  Во вкусах пользователей Москвы и Петербурга больше общего чем различий. 